# Preprocessing

In [1]:
# # You may need to run those in your enviroment terminal.

%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import spacy
import nltk
from nltk.corpus import wordnet
import random
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datasets import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import torch


nltk.download('wordnet')
nltk.download('omw-1.4')

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

c:\Users\hanee\anaconda3\envs\sentiment-nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hanee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hanee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Creating data frame of the data and assigning them labels

In [3]:
texts = []
labels = []

base_path = Path('review_polarity/txt_sentoken')
pos_path = base_path / 'pos'
neg_path = base_path / 'neg'

# Assign label 1
if pos_path.exists():
    for file in pos_path.glob('*.txt'):
        with open(file, 'r', encoding = 'utf-8') as f:
            texts.append(f.read())
            labels.append(1)

# Assign label 0
if neg_path.exists():
    for file in neg_path.glob('*.txt'):
        with open(file, 'r', encoding = 'utf-8') as f:
            texts.append(f.read())
            labels.append(0)

df = pd.DataFrame({
    'label': labels,
    'text': texts
})

df.tail()

,label,text
1995,0,"if anything , "" stigmata "" should be taken as ..."
1996,0,"john boorman's "" zardoz "" is a goofy cinematic..."
1997,0,the kids in the hall are an acquired taste . \...
1998,0,there was a time when john carpenter was a gre...
1999,0,two party guys bob their heads to haddaway's d...


In [4]:
df[df.duplicated()]

,label,text


No duplicates were found

In [5]:
def spacy_pos_to_wordnet_pos(spacy_pos):
    if spacy_pos.startswith('N'):
        return wordnet.NOUN
    elif spacy_pos.startswith('V'):
        return wordnet.VERB
    elif spacy_pos.startswith('J'):
        return wordnet.ADJ
    elif spacy_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default fallback

In [6]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    synonyms.discard(word)      # Remove the original word to avoid replacement with itself
    return list(synonyms)

In [7]:
def augment_text_with_synonyms(tokens, pos_tags, synonym_probability = 0.2):
    augmented_tokens = []

    for token, pos_tag in zip(tokens, pos_tags):
        if random.random() < synonym_probability:
            if pos_tag in ['n', 'v', 'a']:   #nouns adjectives and verbs
                synonyms = get_synonyms(token)
                if synonyms:
                    new_word = random.choice(synonyms)
                    augmented_tokens.append(new_word)
                    continue
        augmented_tokens.append(token)         # Add the original word if no augmentation is done

    return augmented_tokens

In [8]:
def apply_augmentation_to_dataset(word_tokens, pos_tags, texts, labels, sentence_tokens, synonym_probability = 0.2):
    all_word_tokens = word_tokens.copy()
    all_texts = texts.copy()  # Original text is preserved
    all_labels = labels.copy()
    all_sentence_tokens = sentence_tokens.copy()
    all_pos_tags = pos_tags.copy()

    for tokens, pos, label, sentence, text in zip(word_tokens, pos_tags, labels, sentence_tokens, texts):
        augmented_tokens = augment_text_with_synonyms(tokens, pos, synonym_probability)

        all_word_tokens.append(augmented_tokens)
        all_texts.append(text)  # Keep the original text
        all_labels.append(label)
        all_pos_tags.append(pos)
        all_sentence_tokens.append(sentence)  # Sentence tokens are not augmented

    return all_word_tokens, all_pos_tags, all_texts, all_labels, all_sentence_tokens

In [9]:
def apply_lemmatization(tokens):
    return [token.lemma_ for token in nlp(' '.join(tokens))]

In [10]:
def apply_stemming(text):

    words = text.split()
    return ' '.join([PorterStemmer().stem(word) for word in words])

In [11]:
def visualize_tf_idf_heatmap(tfidf_matrix, feature_names, n_top_features = 20, n_top_docs = 10):
    # Get the top features by summing TF-IDF scores across documents
    tfidf_array = tfidf_matrix.toarray()
    feature_importance = np.sum(tfidf_array, axis = 0)
    top_feature_indices = np.argsort(feature_importance)[-n_top_features:][::-1]
    top_features = [feature_names[i] for i in top_feature_indices]

    # Get the top documents by summing TF-IDF scores across features
    doc_importance = np.sum(tfidf_array, axis=1)
    top_doc_indices = np.argsort(doc_importance)[-n_top_docs:][::-1]

    # Extract the submatrix for visualization
    sub_matrix = tfidf_array[np.ix_(top_doc_indices, top_feature_indices)]

    plt.figure(figsize = (12, 8))
    sns.heatmap(
        sub_matrix,
        annot = True,          # Show values in cells
        fmt = '.3f',           # Format with 3 decimal places
        cmap = 'YlGnBu',       # Better colormap
        xticklabels = top_features,
        yticklabels = range(n_top_docs)
    )
    plt.title(f'TF-IDF Heatmap (Top {n_top_features} Features, First {n_top_docs} Documents)')
    plt.xlabel('Words')
    plt.ylabel('Documents')
    plt.xticks(rotation = 45, ha = 'right')
    plt.tight_layout()
    plt.savefig('tfidf_heatmap.png')
    plt.close()

    # Bar chart of top features across the corpus
    plt.figure(figsize=(12, 6))
    top_features_scores = [feature_importance[i] for i in top_feature_indices]
    plt.bar(top_features, top_features_scores)
    plt.title('Top TF-IDF Features Across All Documents')
    plt.xlabel('Features')
    plt.ylabel('Sum of TF-IDF Scores')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('tfidf_top_features.png')
    plt.close()

In [12]:
texts = df['text'].tolist()
labels = df['label'].tolist()
word_tokens = []
sentence_tokens = []
pos_tags = []

for text in texts:
    doc = nlp(text)
    sentence_tokens.append([sent.text.strip() for sent in doc.sents])
    tokens = [token for token in doc if not token.is_punct and not token.is_stop and not token.is_space]
    word_tokens.append([token.text.lower() for token in tokens])
    pos_tags.append([spacy_pos_to_wordnet_pos(token.tag_) for token in tokens])

lemmatized_tokens = [apply_lemmatization(t) for t in word_tokens]

word_tokens, pos_tags, texts, labels, sentence_tokens = apply_augmentation_to_dataset(
    lemmatized_tokens, pos_tags, texts, labels, sentence_tokens
)

joined_texts = [' '.join(tokens) for tokens in word_tokens]

augmented_df = pd.DataFrame({
    'label': labels,
    'text': joined_texts,
    'word_tokens': word_tokens
})

augmented_df = augmented_df.sample(frac = 1, random_state = 42).reset_index(drop = True)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    augmented_df['text'], augmented_df['label'], test_size = 0.2, random_state = 42
)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
vectorizer.fit(train_texts)
xtrain_tfidf = vectorizer.transform(train_texts)
xtest_tfidf = vectorizer.transform(test_texts)

tfidf_matrix = vectorizer.fit_transform(joined_texts)
feature_names = vectorizer.get_feature_names_out()

print(f"\nTF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Number of features: {len(feature_names)}")
visualize_tf_idf_heatmap(tfidf_matrix, feature_names)


TF-IDF Matrix Shape: (4000, 42868)
Number of features: 42868


In [13]:
print("augmented_df.head():")
display(augmented_df.head())

print("\ndf.head():")
display(df.head())

augmented_df.head():


,label,text,word_tokens
0,1,harmless silly fun comedy dim witte wrestle fa...,"[harmless, silly, fun, comedy, dim, witte, wre..."
1,0,point movie staging opera go completely wrong ...,"[point, movie, staging, opera, go, completely,..."
2,1,sick life death bob flanagan supermasochist fe...,"[sick, life, death, bob, flanagan, supermasoch..."
3,0,everybody film think alicia documentary see cr...,"[everybody, film, think, alicia, documentary, ..."
4,1,lisa cholodenko heights art intelligent quiet ...,"[lisa, cholodenko, heights, art, intelligent, ..."



df.head():


,label,text
0,1,films adapted from comic books have had plenty...
1,1,every now and then a movie comes along from a ...
2,1,you've got mail works alot better than it dese...
3,1,""" jaws "" is a rare film that grabs your atten..."
4,1,moviemaking is a lot like being the general ma...


In [14]:
print("augmented_df.tail():")
display(augmented_df.tail())

print("\ndf.tail():")
display(df.tail())

augmented_df.tail():


,label,text,word_tokens
3995,0,understand clich hell earth truly mean recentl...,"[understand, clich, hell, earth, truly, mean, ..."
3996,0,1954 japanese monster film godzilla transform ...,"[1954, japanese, monster, film, godzilla, tran..."
3997,1,verdict spine chilling drama horror maestro st...,"[verdict, spine, chilling, drama, horror, maes..."
3998,0,midway anaconda documentary filmmaker terri fl...,"[midway, anaconda, documentary, filmmaker, ter..."
3999,0,starship trooper regretful moving_picture mean...,"[starship, trooper, regretful, moving_picture,..."



df.tail():


,label,text
1995,0,"if anything , "" stigmata "" should be taken as ..."
1996,0,"john boorman's "" zardoz "" is a goofy cinematic..."
1997,0,the kids in the hall are an acquired taste . \...
1998,0,there was a time when john carpenter was a gre...
1999,0,two party guys bob their heads to haddaway's d...


In [15]:
print("\ndf dimentions:")
display(df.shape)

print("\naugmented_df dimentions:")
display(augmented_df.shape)


df dimentions:


(2000, 2)


augmented_df dimentions:


(4000, 3)

# Modelling - Classifiers

####        •    ML -> Logistic Regression, Naive Bayes, SVM, Decision tree, and Random Forest.

In [16]:
def train_model(classifier, feature_vector_train, train_labels, feature_vector_test, test_labels, is_neural_net=False):
    classifier.fit(feature_vector_train, train_labels)

    train_predictions = classifier.predict(feature_vector_train)
    test_predictions = classifier.predict(feature_vector_test)
    train_accuracy = metrics.accuracy_score(train_labels, train_predictions)
    test_accuracy = metrics.accuracy_score(test_labels, test_predictions)

    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

#### 1. Logistic Regression

In [17]:
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 98.06%
Test Accuracy: 91.38%


#### 2. Naive Bayes

In [18]:
accuracy = train_model(naive_bayes.MultinomialNB(alpha = 0.0001), xtrain_tfidf, train_labels, xtest_tfidf,test_labels)

Train Accuracy: 99.94%
Test Accuracy: 92.12%


#### 3. SVM

In [19]:
accuracy = train_model(svm.SVC(kernel = 'linear', C = 1.0), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 99.62%
Test Accuracy: 95.50%


In [20]:
accuracy = train_model(svm.SVC(kernel = 'rbf', C = 1.0), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 99.94%
Test Accuracy: 96.38%


In [21]:
accuracy = train_model(svm.SVC(kernel = 'poly', C = 1.0, degree = 3), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 100.00%
Test Accuracy: 92.88%


#### 4. Decision Tree

In [22]:
accuracy = train_model(DecisionTreeClassifier(), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 100.00%
Test Accuracy: 79.75%


In [23]:
accuracy = train_model(RandomForestClassifier(), xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

Train Accuracy: 100.00%
Test Accuracy: 94.88%


#### • DL -> BERT

In [28]:


# Use same train/test split for BERT
bert_train_texts = train_texts.tolist()
bert_test_texts = test_texts.tolist()
bert_train_labels = train_labels.tolist()
bert_test_labels = test_labels.tolist()

bert_train_ds = Dataset.from_dict({"text": bert_train_texts, "label": bert_train_labels})
bert_test_ds = Dataset.from_dict({"text": bert_test_texts, "label": bert_test_labels})


In [29]:


# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the datasets
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

bert_train_ds = bert_train_ds.map(tokenize_function, batched=True)
bert_test_ds = bert_test_ds.map(tokenize_function, batched=True)

# Set PyTorch format
bert_train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
bert_test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map: 100%|██████████| 800/800 [00:04<00:00, 181.77 examples/s]


In [33]:

# Load pre-trained BERT for classification
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define evaluation metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}


c:\Users\hanee\anaconda3\envs\sentiment-nlp\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\hanee\anaconda3\envs\sentiment-nlp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hanee\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

In [34]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_ds,
    eval_dataset=bert_test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

                                                   
 16%|█▌        | 124/800 [52:24<1:28:46,  7.88s/it]

{'loss': 0.6949, 'learning_rate': 4.96875e-05, 'epoch': 0.01}


                                                   
 16%|█▌        | 124/800 [52:34<1:28:46,  7.88s/it]

{'loss': 0.7136, 'learning_rate': 4.937500000000001e-05, 'epoch': 0.03}


                                                   
 16%|█▌        | 124/800 [52:43<1:28:46,  7.88s/it]

...

                                                     
 16%|█▌        | 124/800 [1:22:21<1:28:46,  7.88s/it]

{'loss': 0.3122, 'learning_rate': 0.0, 'epoch': 2.0}










































































































































































































                                                     

                                           
 16%|█▌        | 124/800 [1:23:22<1:28:46,  7.88s/it]

                                                     
100%|██████████| 1600/1600 [31:06<00:00,  1.17s/it]t]

{'eval_loss': 0.5407305359840393, 'eval_accuracy': 0.845, 'eval_runtime': 61.0636, 'eval_samples_per_second': 13.101, 'eval_steps_per_second': 3.275, 'epoch': 2.0}
{'train_runtime': 1866.4234, 'train_samples_per_second': 3.429, 'train_steps_per_second': 0.857, 'train_loss': 0.5134868958964944, 'epoch': 2.0}


TrainOutput(global_step=1600, training_loss=0.5134868958964944, metrics={'train_runtime': 1866.4234, 'train_samples_per_second': 3.429, 'train_steps_per_second': 0.857, 'train_loss': 0.5134868958964944, 'epoch': 2.0})

In [35]:
# Evaluate BERT model on test set
metrics = trainer.evaluate()
print(f"BERT Classification Accuracy: {metrics['eval_accuracy']:.4f}")


100%|██████████| 200/200 [00:48<00:00,  4.16it/s]

BERT Classification Accuracy: 0.8450
